<a href="https://colab.research.google.com/github/parkminhyung/python-code-for-finance/blob/main/portfolio%20optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- load required packages

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

- set parameters
- we create 5000 random portfolios
- the actual risk-free rate in real market is not "0", but we supposed that its value is 0
- init_asset is amount of asset

In [ ]:
tickers = ['AAPL','MSFT','TSLA','AMZN','GOOG','2222.SR']
num_port = 5000 #the number of random portfolio
rf = 0 #risk-free rate, the actual risk free rate is not "0",
init_asset = 100000

- create empty data frame and put stock value into the dataframe
- preprocess na value

In [ ]:
df = pd.DataFrame()
for ticker in tickers :
  data = yf.download(
    ticker,
    start  = "2020-01-01",
    end = "2023-01-01")
  price = data[["Adj Close"]]
  price = pd.DataFrame(np.log(price["Adj Close"]) - np.log(price["Adj Close"].shift(1)))
  price.columns = [ticker]
  df = pd.concat([df,price],axis=1)

df = (np.round(df,5)).dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


- calculate portfolios' return and covariance

In [ ]:
mean_return = df.mean()
cov_mat = df.cov()*252

- Create Random weights Portfolio

In [ ]:
all_wts = pd.DataFrame(np.zeros((num_port, df.shape[1])))
all_wts.columns = [ticker + "_weights"  for ticker in tickers]
for i in range(num_port) :
  all_wts.iloc[i] = np.random.uniform(low=0.0, high=1.0, size=df.shape[1])
  all_wts.iloc[i] = all_wts.iloc[i]/np.sum(all_wts.iloc[i])
all_wts

,AAPL_weights,MSFT_weights,TSLA_weights,AMZN_weights,GOOG_weights,2222.SR_weights
0,0.191262,0.154677,0.044238,0.255418,0.004263,0.350143
1,0.285482,0.047652,0.162000,0.208562,0.191046,0.105257
2,0.161606,0.088183,0.176007,0.177540,0.192436,0.204228
3,0.250711,0.012598,0.144337,0.176377,0.172898,0.243080
4,0.229632,0.096860,0.191625,0.184867,0.085406,0.211609
...,...,...,...,...,...,...
4995,0.112593,0.210867,0.104104,0.094156,0.105602,0.372678
4996,0.387388,0.011257,0.065280,0.036528,0.150681,0.348866
4997,0.000705,0.193842,0.280388,0.220835,0.016180,0.288051
4998,0.174549,0.247354,0.194634,0.049424,0.243212,0.090826


- Create return, risk, sharpe-ratio columns and calculate those values

In [ ]:
all_wts["return"], all_wts["risk"], all_wts["sharpe"] = (np.zeros((num_port, 1)) for _ in range(3))
for i in range(num_port) :
  all_wts["return"][i] = pow((1+np.array(all_wts.iloc[i,:len(tickers)]).dot(mean_return)),252)-1
  all_wts["risk"][i] = np.sqrt(np.array(all_wts.iloc[i,:len(tickers)]).dot(cov_mat.dot(np.array(all_wts.iloc[i,:len(tickers)]))))
all_wts["sharpe"] = (all_wts["return"]-rf)/all_wts["risk"]
all_wts

,AAPL_weights,MSFT_weights,TSLA_weights,AMZN_weights,GOOG_weights,2222.SR_weights,return,risk,sharpe
0,0.191262,0.154677,0.044238,0.255418,0.004263,0.350143,0.277515,0.243549,1.139462
1,0.285482,0.047652,0.162000,0.208562,0.191046,0.105257,0.391679,0.331413,1.181847
2,0.161606,0.088183,0.176007,0.177540,0.192436,0.204228,0.386197,0.307998,1.253894
3,0.250711,0.012598,0.144337,0.176377,0.172898,0.243080,0.372146,0.290214,1.282315
4,0.229632,0.096860,0.191625,0.184867,0.085406,0.211609,0.415239,0.314496,1.320333
...,...,...,...,...,...,...,...,...,...
4995,0.112593,0.210867,0.104104,0.094156,0.105602,0.372678,0.313843,0.249584,1.257464
4996,0.387388,0.011257,0.065280,0.036528,0.150681,0.348866,0.319480,0.251191,1.271861
4997,0.000705,0.193842,0.280388,0.220835,0.016180,0.288051,0.468904,0.330213,1.420003
4998,0.174549,0.247354,0.194634,0.049424,0.243212,0.090826,0.398324,0.345082,1.154286


- "tang" is tangency portfolio that has the largest sharpe ratio value
- "risk-min" is minimized risk portfolio that has the smallest risk value

In [ ]:
tang = all_wts.iloc[all_wts["sharpe"].idxmax()]
risk_min = all_wts.iloc[all_wts["risk"].idxmin()]


- show weights and allocation of tangency portfolio

In [ ]:
print( "[ tangency portfolio % ]" ,"\n" ,tang.iloc[:len(tickers)],"\n","\n",
"[ asset allocation $]","\n",">> Initial Asset : ",init_asset,"\n",tang.iloc[:len(tickers)] * init_asset)


[ tangency portfolio % ] 
 AAPL_weights       0.015926
MSFT_weights       0.016397
TSLA_weights       0.397181
AMZN_weights       0.058858
GOOG_weights       0.074652
2222.SR_weights    0.436986
Name: 3628, dtype: float64 
 
 [ asset allocation $] 
 >> Initial Asset :  100000 
 AAPL_weights        1592.634763
MSFT_weights        1639.701187
TSLA_weights       39718.140202
AMZN_weights        5885.771805
GOOG_weights        7465.170128
2222.SR_weights    43698.581914
Name: 3628, dtype: float64


- show weights and allocation of risk-min portfolio

In [ ]:
#global minimize portfolio
print( "[ global minimize portfolio %]" ,"\n",risk_min.iloc[:len(tickers)],"\n","\n",
"[ asset allocation $]","\n",">> Initial Asset : ",init_asset,"\n",risk_min.iloc[:len(tickers)] * init_asset)



[ global minimize portfolio %] 
 AAPL_weights       0.017336
MSFT_weights       0.025065
TSLA_weights       0.001112
AMZN_weights       0.309600
GOOG_weights       0.117245
2222.SR_weights    0.529641
Name: 2116, dtype: float64 
 
 [ asset allocation $] 
 >> Initial Asset :  100000 
 AAPL_weights        1733.585851
MSFT_weights        2506.530542
TSLA_weights         111.225818
AMZN_weights       30960.019233
GOOG_weights       11724.498651
2222.SR_weights    52964.139905
Name: 2116, dtype: float64


- Visualize random portfolio and create annotations (global minimized portfolio and tangency portfolio)

In [ ]:
ann_min = go.layout.Annotation(
    x=risk_min['risk'],
    y=risk_min['return'],
    text="Minimum Variance Portfolio",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=0
)

ann_tang = go.layout.Annotation(
    x=tang['risk'],
    y=tang['return'],
    text="Tangency Portfolio",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=0
)

# Define layout for titles
title1 = {
    'text': "<b> Portfolio Optimization and Efficient Frontier </b>",
    'xref': "paper",
    'yref': "paper",
    'xanchor': "center",
    'yanchor': "bottom",
    'align': "center",
    'x': 0.5,
    'y': 1,
    'showarrow': False
}

title2 = {
    'text': "<b> Portfolio Weights </b>",
    'xref': "paper",
    'yref': "paper",
    'xanchor': "center",
    'yanchor': "bottom",
    'align': "center",
    'x': 0.5,
    'y': 1,
    'showarrow': False
}

# Create a subplot with two figures
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=all_wts['risk'],
    y=all_wts['return'],
    mode='markers',
    marker=dict(
        color=all_wts['sharpe'],
        colorscale='Viridis',
        showscale=True
    ),
    showlegend=False
))

fig1.add_trace(go.Scatter(
    x=[risk_min['risk']],
    y=[risk_min['return']],
    mode='markers',
    marker=dict(color="#A52929"),
))

fig1.add_trace(go.Scatter(
    x=[tang['risk']],
    y=[tang['return']],
    mode='markers',
    marker=dict(color="#A52929"),
))

fig1.update_xaxes(title="σ", zeroline=False)
fig1.update_yaxes(title="µ", zeroline=False)
fig1.update_layout(annotations=[title1, ann_min, ann_tang], coloraxis_colorbar=dict(title="Sharpe Ratio"))
fig1.show()

- build weights plots from portfolios

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Bar(
    x= risk_min.iloc[:len(tickers)].index,
    y= risk_min.iloc[:len(tickers)],
    name="Minimum Weights",
    marker=dict(color='#3246AB')
))

fig2.add_trace(go.Bar(
    x=tang.iloc[:len(tickers)].index,
    y=tang.iloc[:len(tickers)],
    name="Tangency Weights",
    marker=dict(color='#12B7C3')
))

fig2.update_layout(annotations=[title2], yaxis=dict(title="Weights"), barmode="group")
fig2.show()

- merge two plots by making subplots

In [ ]:
# Create subplots
subplots = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("<b> Portfolio Optimization and Efficient Frontier </b>", "<b> Portfolio Weights </b>"))
for trace in fig1.data : subplots.add_trace(trace, row=1,col=1)
for trace in fig2.data : subplots.add_trace(trace, row=2,col=1)
subplots.update_layout(title_x=0.5, title_y=1, margin=dict(b=0.1))
subplots.show()